# Spark DataFrame samples

Python API Documentation:  https://spark.apache.org/docs/latest/api/python/index.html

In [1]:
# Prereqquisites
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
# Spark Session and Context
spark = SparkSession.builder.master("local").getOrCreate()
sc = spark.sparkContext
print("Spark Version: ", spark.version)

Spark Version:  3.5.0


In [3]:
import numpy as np
import pandas as pd
import matplotlib as plt
print("NumPy version: ", np.__version__)
print("Pandas version: ", pd.__version__)
print("Matplotlib version: ", plt.__version__)

NumPy version:  1.24.4
Pandas version:  2.0.3
Matplotlib version:  3.8.0


### Load dataframe from CSV file

In [28]:
# Define the path to the data file within the "sparkdata" volume
data_path = "data/taxi_zone_lookup.csv"

# Load the data into a PySpark DataFrame
#df = spark.read.csv(data_path, header=True, inferSchema=True)

# Alternative Way to read:
# Default format is Parquet, if .format() not specified
"""
df = (spark.read
      .format("csv")
      .option("HEADER", "true")
      .option("InferSchEMA" ,"true")
      .load(data_path))
"""

# Another Alternative Way to read:
df = (spark.read
      .format("csv")
      .option("HEADER", "true")
      .option("InferSchEMA" ,"true")
      .option("path", "data/taxi_zone_lookup.csv")
      .load())
      
# Show the first few rows of the DataFrame
df.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [29]:
# Use the display function
display(df)

DataFrame[LocationID: int, Borough: string, Zone: string, service_zone: string]

In [30]:
print("Numer of rows: ", df.count())
df.printSchema()

Numer of rows:  265
root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



## Create a Dataframe manually and group

In [31]:
# Create Dataframe
df_persons = spark.createDataFrame([("Brooke", 20), ("Denny", 31), ("Jules", 30), ("TD", 35), 
                                    ("Brooke", 25)], ["name", "age"])

df_persons.show() 

+------+---+
|  name|age|
+------+---+
|Brooke| 20|
| Denny| 31|
| Jules| 30|
|    TD| 35|
|Brooke| 25|
+------+---+



In [32]:
# Group names and compute average
df_ave = df_persons.groupBy("name").agg(avg("age"))
df_ave.show()

+------+--------+
|  name|avg(age)|
+------+--------+
|Brooke|    22.5|
| Jules|    30.0|
|    TD|    35.0|
| Denny|    31.0|
+------+--------+



## Create Dataframe with Schema

In [33]:
from pyspark.sql.types import *

In [34]:
# Create Schema
schema =  "`Id` INT,`First` STRING,`Last` STRING,`Url` STRING,`Published` STRING,`Hits` INT,`Campaigns` ARRAY<STRING>"
print(schema)

`Id` INT,`First` STRING,`Last` STRING,`Url` STRING,`Published` STRING,`Hits` INT,`Campaigns` ARRAY<STRING>


In [35]:
# Create static data
data = [[1, "Jules", "Damji", "https://tinyurl.1", "1/4/2016", 4535, ["twitter",
"LinkedIn"]],
       [2, "Brooke","Wenig", "https://tinyurl.2", "5/5/2018", 8908, ["twitter",
"LinkedIn"]],
       [3, "Denny", "Lee", "https://tinyurl.3", "6/7/2019", 7659, ["web",
"twitter", "FB", "LinkedIn"]],
       [4, "Tathagata", "Das", "https://tinyurl.4", "5/12/2018", 10568, 
["twitter", "FB"]],
       [5, "Matei","Zaharia", "https://tinyurl.5", "5/14/2014", 40578, ["web",
"twitter", "FB", "LinkedIn"]],
       [6, "Reynold", "Xin", "https://tinyurl.6", "3/2/2015", 25568, 
["twitter", "LinkedIn"]]
      ]

In [36]:
# Create Dataframe
df_blogs = spark.createDataFrame(data, schema)

df_blogs.show()

+---+---------+-------+-----------------+---------+-----+--------------------+
| Id|    First|   Last|              Url|Published| Hits|           Campaigns|
+---+---------+-------+-----------------+---------+-----+--------------------+
|  1|    Jules|  Damji|https://tinyurl.1| 1/4/2016| 4535| [twitter, LinkedIn]|
|  2|   Brooke|  Wenig|https://tinyurl.2| 5/5/2018| 8908| [twitter, LinkedIn]|
|  3|    Denny|    Lee|https://tinyurl.3| 6/7/2019| 7659|[web, twitter, FB...|
|  4|Tathagata|    Das|https://tinyurl.4|5/12/2018|10568|       [twitter, FB]|
|  5|    Matei|Zaharia|https://tinyurl.5|5/14/2014|40578|[web, twitter, FB...|
|  6|  Reynold|    Xin|https://tinyurl.6| 3/2/2015|25568| [twitter, LinkedIn]|
+---+---------+-------+-----------------+---------+-----+--------------------+



In [37]:
# Get Schema definition
print("Schema:\n", df_blogs.schema)

Schema:
 StructType([StructField('Id', IntegerType(), True), StructField('First', StringType(), True), StructField('Last', StringType(), True), StructField('Url', StringType(), True), StructField('Published', StringType(), True), StructField('Hits', IntegerType(), True), StructField('Campaigns', ArrayType(StringType(), True), True)])


## Columns

In [38]:
# Select Hits Column and multiply by 2
df_blogs.select(expr("Hits") * 2).show()

+----------+
|(Hits * 2)|
+----------+
|      9070|
|     17816|
|     15318|
|     21136|
|     81156|
|     51136|
+----------+



In [39]:
# Add togeher Id and Hits columns
df_blogs.select(expr("Id") + expr("Hits")).show(truncate=False)

+-----------+
|(Id + Hits)|
+-----------+
|4536       |
|8910       |
|7662       |
|10572      |
|40583      |
|25574      |
+-----------+



In [40]:
# Add a Column
df_blogs.withColumn("Big Hitters", (expr("Hits") > 15000)).show()

+---+---------+-------+-----------------+---------+-----+--------------------+-----------+
| Id|    First|   Last|              Url|Published| Hits|           Campaigns|Big Hitters|
+---+---------+-------+-----------------+---------+-----+--------------------+-----------+
|  1|    Jules|  Damji|https://tinyurl.1| 1/4/2016| 4535| [twitter, LinkedIn]|      false|
|  2|   Brooke|  Wenig|https://tinyurl.2| 5/5/2018| 8908| [twitter, LinkedIn]|      false|
|  3|    Denny|    Lee|https://tinyurl.3| 6/7/2019| 7659|[web, twitter, FB...|      false|
|  4|Tathagata|    Das|https://tinyurl.4|5/12/2018|10568|       [twitter, FB]|      false|
|  5|    Matei|Zaharia|https://tinyurl.5|5/14/2014|40578|[web, twitter, FB...|       true|
|  6|  Reynold|    Xin|https://tinyurl.6| 3/2/2015|25568| [twitter, LinkedIn]|       true|
+---+---------+-------+-----------------+---------+-----+--------------------+-----------+



In [41]:
# Concatenate columns
df_blogs.withColumn("AuthorsId", (concat(expr("First"), expr("Last"), expr("Id")))).select(expr("AuthorsId")).show()

+-------------+
|    AuthorsId|
+-------------+
|  JulesDamji1|
| BrookeWenig2|
|    DennyLee3|
|TathagataDas4|
|MateiZaharia5|
|  ReynoldXin6|
+-------------+



In [42]:
# Original
df_blogs.show()

+---+---------+-------+-----------------+---------+-----+--------------------+
| Id|    First|   Last|              Url|Published| Hits|           Campaigns|
+---+---------+-------+-----------------+---------+-----+--------------------+
|  1|    Jules|  Damji|https://tinyurl.1| 1/4/2016| 4535| [twitter, LinkedIn]|
|  2|   Brooke|  Wenig|https://tinyurl.2| 5/5/2018| 8908| [twitter, LinkedIn]|
|  3|    Denny|    Lee|https://tinyurl.3| 6/7/2019| 7659|[web, twitter, FB...|
|  4|Tathagata|    Das|https://tinyurl.4|5/12/2018|10568|       [twitter, FB]|
|  5|    Matei|Zaharia|https://tinyurl.5|5/14/2014|40578|[web, twitter, FB...|
|  6|  Reynold|    Xin|https://tinyurl.6| 3/2/2015|25568| [twitter, LinkedIn]|
+---+---------+-------+-----------------+---------+-----+--------------------+



## Rows

In [43]:
from pyspark.sql import Row

In [79]:
# Create a Row object
blog_row = Row(6, "Reynold", "Xin", "https://tinyurl.6", 255568, "3/2/2015", ["twitter", "LinkedIn"])
# Access using element with 0-counted index
print(blog_row[2])

Xin


In [44]:
# Create a datafame with ROws
rows = [Row("John Smith", "WA", 36), Row("Jane Doe", "CA", 62)]
df_people = spark.createDataFrame(rows, ["Authors", "State", "Age"])
df_people.show()

+----------+-----+---+
|   Authors|State|Age|
+----------+-----+---+
|John Smith|   WA| 36|
|  Jane Doe|   CA| 62|
+----------+-----+---+

